In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

In [4]:
df = pd.read_csv('./data/train.csv')[['Age','Fare','SibSp','Parch','Survived']]

In [5]:
df.dropna(inplace=True)

In [6]:
df.head()

,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [7]:
df['family'] = df['SibSp'] + df['Parch']

In [8]:
df.head()

,Age,Fare,SibSp,Parch,Survived,family
0,22.0,7.2500,1,0,0,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,0,0,1,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [9]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [10]:
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


In [11]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [14]:
X_train.head()

,Age,Fare,family
37,21.0,8.0500,0
58,5.0,27.7500,3
57,28.5,7.2292,0
94,59.0,7.2500,0
209,40.0,31.0000,0


In [15]:
# Without binarization

clf = DecisionTreeClassifier()

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_test,y_pred)

0.6465116279069767

In [16]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

np.float64(0.6471048513302036)

In [17]:
# Applying Binarization

from sklearn.preprocessing import Binarizer

In [18]:
trf = ColumnTransformer([
    ('bin',Binarizer(copy=False),['family'])
],remainder='passthrough')

In [19]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [20]:
pd.DataFrame(X_train_trf,columns=['family','Age','Fare'])

,family,Age,Fare
0,0.0,21.0,8.0500
1,1.0,5.0,27.7500
2,0.0,28.5,7.2292
3,0.0,59.0,7.2500
4,0.0,40.0,31.0000
...,...,...,...
494,1.0,46.0,61.1750
495,0.0,25.0,13.0000
496,0.0,41.0,134.5000
497,1.0,33.0,20.5250


In [21]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)

accuracy_score(y_test,y_pred2)

0.6232558139534884

In [22]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

np.float64(0.6317683881064162)